In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import pickle
import os, re
import nltk
BASE_DIR = '../input/'
LABELED_TRAIN_DF = BASE_DIR + 'labeledTrainData.tsv'
UNLABELED_TRAIN_DF = BASE_DIR + 'unlabeledTrainData.tsv'
TEST_DF = BASE_DIR + 'testData.tsv'
print(os.listdir(BASE_DIR))

['word2vec_model_300dim_40minwords_10context', 'sampleSubmission.csv', 'labeledTrainData.tsv', 'test_submission.csv', 'testData.tsv', 'unlabeledTrainData.tsv']


In [2]:
labeled_train = pd.read_csv(LABELED_TRAIN_DF, header = 0, delimiter = '\t', quoting=3)
unlabeled_train = pd.read_csv(UNLABELED_TRAIN_DF, header = 0, delimiter = '\t', quoting=3)
test = pd.read_csv(TEST_DF, header = 0, delimiter = '\t', quoting=3)
print "Read %d labeled train reviews, %d unlabeled train reviews, " \
          "and %d test reviews" % (labeled_train["review"].size, unlabeled_train["review"].size, test["review"].size)

Read 25000 labeled train reviews, 50000 unlabeled train reviews, and 25000 test reviews


Data leakage

Check if test["sentiment"] is correct 

In [3]:
test["sentiment"] = test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
y_test = test["sentiment"]

Credits: Kaggle tutorial

In [4]:
from nltk.corpus import stopwords

def review_to_clean_review(review, remove_numbers=True, stem_words=False):
    # Function to convert a document to a clean document,
    # optionally removing numbers.  Returns a string.
    #
    # 1. Remove HTML using lxml parser, ranked best by bs4
    review_text = BeautifulSoup(review, "lxml").get_text()
    #
    # https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings#L92
    # https://www.kaggle.com/currie32/the-importance-of-cleaning-text
    #  
    # 2. Remove non-letters and non-numbers
    review_text = re.sub("[^a-zA-Z0-9]", " ", review_text)
    #
    # 3. Optionally remove numbers
    if remove_numbers:
        review_text = re.sub("[0-9]", " ", review_text)
    else:
        review_text = review_text.replace('0', ' zero ')
        review_text = review_text.replace('1', ' one ')
        review_text = review_text.replace('2', ' two ')
        review_text = review_text.replace('3', ' three ')
        review_text = review_text.replace('4', ' four ')
        review_text = review_text.replace('5', ' five ')
        review_text = review_text.replace('6', ' six ')
        review_text = review_text.replace('7', ' seven ')
        review_text = review_text.replace('8', ' eight ')
        review_text = review_text.replace('9', ' nine ')
    
    review_text = review_text.lower()
    
    if stem_words:
        words = review_text.split()
        stemmer = SnowballStemmer('english')
        words = [stemmer.stem(word) for word in words]
        text = " ".join(stemmed_words)
        
    # 6. Return a cleaned string
    return(review_text)

def review_to_wordlist(review, remove_stopwords=False, remove_numbers=True, stem_words=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    # 1. Clean review, split it into words
    words = review_to_clean_review(review, stem_words).split()
    #
    # 2. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 6. Return a list of words
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False, remove_numbers=True, stem_words=False):
    # Function to split a review into parsed sentences. Returns a
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence, \
                                                        remove_stopwords, remove_numbers, stem_words))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [5]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# Initialize an empty list of sentences
sentences = []
# Parsing sentences from training set
counter = 0.
for review in labeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer, remove_numbers=False)
    if counter % 5000. == 0.:
        print "Labled train review %d of %d" % (counter, len(labeled_train["review"]))
    counter = counter + 1.

counter = 0.
# Parsing sentences from unlabeled set
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer, remove_numbers=False)
    if counter % 5000. == 0.:
        print "Unlabled train review %d of %d" % (counter, len(unlabeled_train["review"]))
    counter = counter + 1.

Labled train review 0 of 25000


/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:272: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Labled train review 5000 of 25000
Labled train review 10000 of 25000
Labled train review 15000 of 25000
Labled train review 20000 of 25000
Unlabled train review 0 of 50000


/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Unlabled train review 5000 of 50000
Unlabled train review 10000 of 50000
Unlabled train review 15000 of 50000
Unlabled train review 20000 of 50000


...

/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:272: UserWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Unlabled train review 35000 of 50000


/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.youtube.com/watch?v=a0KSqelmgN8"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Unlabled train review 40000 of 50000
Unlabled train review 45000 of 50000


/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://jake-weird.blogspot.com/2007/08/beneath.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Training the Word2Vec model

In [6]:
# Import the built-in logging module and configure it so that Word2Vec
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', \
                    level=logging.INFO)

from gensim.models import Word2Vec
EMBEDDING_DIM = 300  # Word vector dimensionality
MIN_WORD_COUNT = 40  # Minimum word count. Kaggle set to 40, to avoid attaching too much importance to individual movie titles.
NUM_THREADS = 4  # Number of threads to run in parallel
CONTEXT = 10  # Context window size
DOWNSAMPLING = 1e-3  # Downsample setting for frequent words
WORD2VEC_MODEL_FILE = BASE_DIR + \
    "word2vec_model_" + \
    str(EMBEDDING_DIM) + "dim_" + \
    str(MIN_WORD_COUNT) + "minwords_" + \
    str(CONTEXT) + "context"

print "Training the Word2Vec model..."
# model = Word2Vec(sentences, workers=NUM_THREADS, \
#                  size=EMBEDDING_DIM, min_count=MIN_WORD_COUNT, \
#                  window=CONTEXT, sample=DOWNSAMPLING, seed=1)
# model.save(WORD2VEC_MODEL_FILE)


Training the Word2Vec model...
